<a href="https://colab.research.google.com/github/t-rex111/fmml_labs/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?

Answer:-In machine learning, the percentage of the validation set, also known as the validation split ratio, is a critical hyperparameter that can significantly impact model training and evaluation. Here's how changing the percentage of the validation set affects the accuracy of the validation set, and what happens when you increase or reduce it:

Increasing the Percentage of Validation Set:

1)Pros:

More Data for Validation: When you allocate a larger percentage of your data to the validation set, you have more data available for model evaluation. This can provide a more reliable estimate of how well your model is likely to perform on unseen data. Reduced Overfitting Risk: A larger validation set can help you detect overfitting more effectively. If your model performs well on a larger validation set, it's a stronger indication of its ability to generalize. 2)Cons:

Less Data for Training: The trade-off of allocating a larger percentage to the validation set is that you have less data available for training your model. This can be a problem, especially if your dataset is small, as it may lead to underfitting. Reducing the Percentage of Validation Set:

1)Pros:

More Data for Training: Allocating a smaller percentage to the validation set means you have more data for training your model. This can be advantageous when working with limited data, as it can potentially lead to better model performance. 2)Cons:

Less Reliable Validation: With a smaller validation set, the accuracy of the validation set may become less reliable. The model's performance on a small validation set may not accurately reflect how it will generalize to unseen data. There's a higher risk of overfitting, where the model may perform well on the validation set but poorly on new data. The choice of the validation split ratio depends on several factors, including the size of your dataset, the complexity of your model, and your goals:

1)Large Datasets: When you have a large dataset, you can afford to allocate a larger percentage to the validation set (e.g., 20-30%) without significantly reducing the amount of data available for training.

2)Small Datasets: In cases where your dataset is small, it's generally advisable to use a smaller validation split (e.g., 10% or less) to ensure you have enough data for training. However, this may result in less reliable validation performance estimates.

3)Cross-Validation: To obtain more robust performance estimates, especially on small datasets, you can use techniques like k-fold cross-validation, which repeatedly splits the data into training and validation sets and averages the results over multiple iterations.

In summary, the percentage of the validation set can impact model training and evaluation. Increasing the percentage provides a more reliable but smaller validation set, while reducing it offers more training data but may result in less reliable validation performance estimates. The choice should strike a balance between these factors based on your specific circumstances and goals.












2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?


Answer:-

The size of the training and validation sets can have a significant impact on how well you can predict the accuracy on the test set using the validation set. Here's how the sizes of these sets can affect predictive performance:

1)Larger Training Set:

a)Pros: With a larger training set, your model has access to more data to learn patterns and generalize better. This can result in a model that is more likely to perform well on the test set. A larger training set can help prevent underfitting. b)Cons: If the validation set is small relative to the training set, it may not provide a strong indication of the model's generalization performance. You might encounter situations where the model performs well on the validation set but poorly on the test set due to limited representation in the validation data. 2)Larger Validation Set:

a)Pros: A larger validation set can provide a more reliable estimate of how well your model is likely to perform on unseen data. It reduces the impact of randomness in the validation process and can help detect overfitting more effectively. b)Cons: If the training set is small relative to the validation set, you may not be fully utilizing your available data for model training, which could lead to suboptimal model performance. 3)Balanced Training and Validation Sets:

a)Pros: Balancing the sizes of the training and validation sets ensures that you have a reasonable amount of data for both training and evaluation. This balance can help provide a more accurate prediction of how well your model will perform on the test set. b)Cons: The downside is that each set is smaller, which can be a limitation when dealing with limited data. 4)Cross-Validation:

a)Pros: Cross-validation, which involves repeatedly splitting the data into training and validation subsets, helps mitigate the impact of the specific split size. It provides a more robust estimate of model performance by averaging over multiple validation sets. b)Cons: Cross-validation can be computationally expensive, especially with a large number of iterations (folds). It can also reduce the amount of data available for model training in each fold. In practice, the choice of the training and validation set sizes should take into account the characteristics of your dataset and your goals. If you have a large dataset, you can afford to allocate larger portions to both the training and validation sets. For smaller datasets, it may be necessary to make a trade-off between a larger validation set for better validation performance estimation and a larger training set for better model training.

Remember that the ultimate goal is to build a model that generalizes well to unseen data, represented by the test set. While the size of the training and validation sets plays a role, other factors like model complexity, hyperparameter tuning, and data quality also influence how accurately you can predict test set performance using the validation set.












3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

1)Large Datasets (thousands to millions of data points):

Training Set: 60-80% of the data. Validation Set: 10-20% of the data. Test Set: 10-20% of the data. In this scenario, you have a large amount of data, so you can afford to allocate a smaller percentage to the validation and test sets while still having sufficient data for training and reliable model evaluation.

2)Medium-Sized Datasets (hundreds to a few thousand data points):

Training Set: 50-70% of the data. Validation Set: 15-20% of the data. Test Set: 15-20% of the data. With medium-sized datasets, you still want a reasonably large training set, but allocating a larger percentage to the validation and test sets helps ensure better model evaluation.

3)Small Datasets (tens to hundreds of data points):

Training Set: 60-80% of the data. Validation Set: 10-20% of the data. Test Set: 10-20% of the data. When working with small datasets, you often need to make a trade-off between having enough data for training and obtaining reliable estimates of model performance. In such cases, cross-validation (e.g., 5-fold or 10-fold) can also be valuable.

4)Very Small Datasets (a few tens of data points or less):

In cases where your dataset is extremely small, you may need to consider alternative approaches such as transfer learning, data augmentation, or leveraging pre-trained models.






Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [13]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [14]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?

Answer:-

Averaging the validation accuracy across multiple splits, commonly referred to as cross-validation, can indeed provide more consistent and reliable results in machine learning compared to a single train-test split. Cross-validation is a technique used to assess the performance and generalization ability of a machine learning model. It helps mitigate the potential impact of randomness and the choice of a single split on the evaluation metrics. Here are some reasons why cross-validation is beneficial for achieving consistent results:

1)Reduced Variance: Cross-validation averages results from multiple folds or splits of the data, which can help reduce the variance associated with a single split. This means that the evaluation metric (e.g., accuracy) is less likely to be influenced by the particular way the data is divided.

2)More Robust Evaluation: By using cross-validation, you get a more comprehensive view of how well your model performs on different subsets of the data. This can help identify whether your model is overfitting to a specific training-test split or if it generalizes well across different subsets.

3)Better Model Selection: Cross-validation is often used for hyperparameter tuning and model selection. Averaging the results from multiple splits can help you choose the best-performing model or hyperparameters more reliably.

4)Enhanced Confidence: Averaging results across multiple splits provides a measure of confidence in the model's performance. It gives you a more realistic estimate of how well your model is likely to perform on unseen data.

Common cross-validation techniques include k-fold cross-validation and stratified k-fold cross-validation, where the data is divided into 'k' subsets (folds), and the model is trained and evaluated 'k' times, with each fold serving as the test set once.

However, it's important to note that cross-validation can be computationally expensive, especially for large datasets or complex models. Additionally, the choice of the number of folds ('k') can impact the results, with higher values of 'k' providing a more accurate but more computationally intensive estimate of performance.

In summary, averaging validation accuracy across multiple splits using cross-validation is a valuable technique in machine learning to obtain more consistent and reliable results, assess model performance, and make informed decisions about model selection and hyperparameter tuning.











2. Does it give more accurate estimate of test accuracy?

Answer:-

Cross-validation, while providing a more robust and reliable estimate of a model's performance compared to a single train-test split, does not necessarily give a more accurate estimate of test accuracy in the sense of being closer to the true performance on unseen, out-of-sample data. Instead, it provides a better estimate of how well your model is likely to perform on average across different random subsets of your data.

Here's why:

1)Closer to True Performance: The primary objective of cross-validation is not to estimate the true test accuracy but rather to estimate how well your model generalizes to unseen data on average. It does this by simulating multiple train-test splits. While it provides a more stable estimate compared to a single split, it may still have some bias or variability.

2)Bias in Cross-Validation: Cross-validation can introduce a small amount of bias in its estimates because each fold excludes a different portion of the data, and the final result is an average of these individual evaluations. This may lead to a slightly optimistic or pessimistic estimate compared to the true performance on unseen data.

3)Variance Reduction: The main advantage of cross-validation is the reduction in variance (i.e., variability) in the estimated performance metric. This reduction in variance makes the estimate more consistent and less sensitive to the random choice of the train-test split, which can be especially helpful when you have limited data.

4)Model Selection and Hyperparameter Tuning: Cross-validation is particularly valuable for comparing different models or tuning hyperparameters because it allows you to assess the average performance of each model or parameter setting across multiple splits, helping you make more informed choices.

In summary, while cross-validation doesn't necessarily provide a more accurate estimate of test accuracy in terms of proximity to the true performance on unseen data, it does provide a more stable and reliable estimate of model performance. It is a valuable tool for model assessment, comparison, and selection in machine learning, and it helps reduce the impact of randomness in the choice of a single train-test split. To get a sense of the true performance on unseen data, you would still need to evaluate your model on a separate, truly held-out test dataset, which is typically not used during model development and tuning.















3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?

Answer:-

In the context of machine learning and cross-validation, the term "number of iterations" typically refers to the number of iterations or folds used in a cross-validation procedure, such as k-fold cross-validation. The number of iterations, in this case, is represented by 'k,' which determines how many times the data is split into training and testing subsets. Higher values of 'k' do not necessarily result in a better estimate, and the choice of 'k' should be made carefully, considering trade-offs.

Here's how the number of iterations (k) can affect the estimate:

Higher k (More Folds):

Pros: Using a higher 'k' value, such as 10-fold or leave-one-out (where each data point is a separate test set), results in a finer-grained estimate of model performance. It reduces the bias in the estimate because each fold is smaller and more representative of the entire dataset. Cons: Using a higher 'k' comes with increased computational cost, as you need to train and evaluate the model 'k' times. It may also lead to higher variance in the estimates because each fold has less data, potentially making the performance estimates noisier. Lower k (Fewer Folds):

Pros: Lower 'k' values, such as 3-fold or 5-fold, are computationally more efficient and require fewer iterations. They provide a smoother estimate but might have higher bias due to larger test sets. Cons: The estimates can be more influenced by the particular random splits in the data. A single outlier fold can have a significant impact on the result. The choice of the appropriate 'k' value depends on various factors:

1)Dataset Size: For small datasets, using a higher 'k' can help ensure that each fold has a reasonable amount of data for training and testing. However, it will increase the computational overhead.

Computational Resources: Consider the available computational resources. Higher 'k' values require more time and memory for each cross-validation run.

2)Desired Precision: If you need a more precise estimate of model performance and are willing to invest computational resources, higher 'k' values can be beneficial. However, if you are looking for a quick estimate, lower 'k' values may suffice.

*3)Data Distribution: If your dataset has a specific structure or is imbalanced, it may influence the choice of 'k.' For instance, when dealing with time series data, you might consider time-based cross-validation techniques.

In practice, common choices for 'k' include 5-fold and 10-fold cross-validation, but the optimal value can vary depending on the specific problem and dataset. It's often a good practice to experiment with different values of 'k' and evaluate how the choice impacts the stability and reliability of your performance estimates.





















4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?

Answer:-

Increasing the number of iterations (folds) in cross-validation can help mitigate the impact of a very small training dataset to some extent, but it may not completely solve the problem if the training dataset is extremely small. The effectiveness of this approach depends on the balance between several factors:

1)Training Set Size: If your training dataset is very small, each fold in cross-validation will also be small when you increase the number of iterations (higher 'k'). While increasing 'k' can provide a better estimate of performance, it doesn't magically increase the amount of training data available in each fold.

2)Computational Resources: As you increase 'k,' you are effectively increasing the number of times your model needs to be trained and evaluated. This can become computationally expensive, and you need to weigh the benefits of more iterations against the increased computational cost.

3)Statistical Significance: Even with a large number of iterations, if each fold has an extremely small training dataset, the performance estimates may still have high variance and be less statistically significant. In such cases, the estimates may be unreliable.

4)Model Complexity: The complexity of the model you're training also plays a role. Extremely complex models may require larger training datasets to learn meaningful patterns. If your model is too complex relative to the dataset size, you may encounter overfitting issues.

5)Data Quality: The quality and representativeness of the small training dataset matter. If it is not representative of the overall data distribution or if it contains noise, increasing 'k' won't necessarily improve the quality of your model's estimates.

In summary, increasing the number of iterations (folds) in cross-validation can be a useful strategy when dealing with a small training dataset to obtain a more stable and reliable estimate of model performance. However, it cannot overcome the fundamental limitation of having very little training data. In such cases, it's crucial to consider other strategies, such as:

1)Data Augmentation: If possible, augment your training data through techniques like data synthesis or augmentation to artificially increase the size of the training dataset.

2)Transfer Learning: Consider using pre-trained models and fine-tuning them on your small dataset. Transfer learning can leverage knowledge learned from larger datasets.

3)Simpler Models: Choose simpler, less complex models that require fewer data points to generalize effectively.

4)Regularization: Apply regularization techniques to prevent overfitting, which can be a significant concern when training on small datasets.

Ultimately, the choice of approach depends on the specific problem, the nature of your dataset, and the available resources. It's often advisable to seek a balance between the number of iterations, model complexity, and data quality to achieve the best results in situations with limited training data.


